### 데이터 크롤링

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pandas as pd
import requests
import time
import csv
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

In [3]:
driver = webdriver.Chrome('/usr/bin/chromedriver', options=options) #

<ipython-input-3-5f35579ee4c2>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/bin/chromedriver', options=options) #


In [4]:
total_url = []
for day in tqdm(range(1, 31)):
    for i in range(1, 21):
        url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={i}&rangeType=PERIOD&orderBy=sim&startDate=2022-11-{day}&endDate=2022-11-{day}&keyword=주간일기챌린지'
        driver.get(url)
        time.sleep(2)
        pages_url = driver.find_elements(By.XPATH, '//*[@id="content"]/section/div[2]/div/div/div[1]/div[1]/a[1]')
        for page_url in pages_url:
            total_url.append(page_url.get_attribute('href'))
        time.sleep(1)

100%|██████████| 30/30 [35:10<00:00, 70.34s/it]


In [5]:
len(total_url)

4200

In [6]:
with open('naverblog_total_url_nov.csv', 'w',newline='') as f: 
    w = csv.writer(f) 
    w.writerow(total_url)

In [ ]:
total_text = []

In [17]:
total_url_ = total_url[1427:]

In [18]:
for url in tqdm(total_url_):
    driver.get(url)
    time.sleep(2)
    driver.switch_to.frame(driver.find_element(By.NAME, 'mainFrame'))
    try:
        t = driver.find_element(By.CLASS_NAME, 'se-main-container')
        total_text.append(t.text)
    except:
        t = ''
        total_text.append(t)
    time.sleep(1)

100%|██████████| 2773/2773 [4:58:59<00:00,  6.47s/it]


In [19]:
len(total_text)

4200

In [ ]:
total_text

In [20]:
with open('naverblog_total_text_nov.csv', 'w',newline='') as f: 
    w = csv.writer(f) 
    w.writerow(total_text)

### 데이터 전처리

In [1]:
import csv
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
f = open('/content/drive/MyDrive/skku/2022-2 culture-nlp/naverblog_total_text.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for text in rdr:
    oct_text = text
f.close()

In [3]:
oct_text.append('')

In [4]:
f = open('/content/drive/MyDrive/skku/2022-2 culture-nlp/naverblog_total_text_nov.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for text in rdr:
    nov_text = text
f.close()

In [5]:
t = oct_text + nov_text

In [6]:
len(t) #총 데이터 개수

8540

In [7]:
stopwords = ['너무', 'ㅋㅋ', '진짜', '주간', '주간일기', '블챌', '그리고', '있는', '하고', 'ㅎㅎ', '많이', '먹고', '정말', '챌린지', '맛있', '내가', '우리', '이번',
             '오늘', '이렇게', '시간', '사진', '그래도', '다시', '요즘', '그냥', '이제', '생각', '하는', '저녁', '집에', 'ㅋㅋㅋ', '주간일기챌린지', '맛있', '나는',
             '블로그', '사람', '열심히', '네이버', '같이', '바로', '좋아', '근데', '역시', '아니', '함께', '해서', '좋은', '있다', '아주', '시작', '그래서', '한다'
             '했는데', '점심', '아침', '엄청', '있어서', '하나', '했다', '갔다']

def cleaning(t):
    t = re.sub('[^ㄱ-ㅎ가-힣 ]', ' ', t) #숫자, 알파벳, 한글 이외 문자 제거 re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣 ]', ' ', t)
    clean_words = []
    words = word_tokenize(t)
    for word in words:
        if word not in stopwords:
            clean_words.append(word)
    t = ' '.join(clean_words)
    return t

In [8]:
clean_t = [cleaning(text) for text in t]

### 데이터 분석
- 키워드 분석
- 명사 키워드 분석
- 동사 출현 횟수 분석
- 시기별 분석
- 연어 분석

 ##### 키워드 분석

In [9]:
! pip install krwordrank
from krwordrank.word import KRWordRank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
wordrank_extractor = KRWordRank(min_count = 5, max_length = 100, verbose = True)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(clean_t, beta, max_iter)

scan vocabs ... 
num vocabs = 144396
done


In [16]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:2000]:
    print('%8s:\t%.4f' % (word, r))

      맛있:	111.4762
      생각:	79.0590
      ㅋㅋ:	75.5292
      엄마:	63.8402
     했는데:	63.2224
     오늘은:	59.9563
      친구:	59.9525
      가을:	59.4971
      좋아:	58.3255
      카페:	58.2173
      보고:	58.1270
      조금:	57.8191
      다음:	57.7194
      일기:	57.4970
      처음:	56.5476
      여기:	56.5024
      가서:	56.4960
      하루:	56.3724
      계속:	55.8542
      시간:	55.7947
      한다:	55.7646
     동영상:	55.6582
      들어:	55.1898
      아이:	55.1761
      먹었:	54.3807
      주차:	53.3831
      사람:	52.3407
      일상:	51.5220
      이런:	50.7830
      다른:	50.7129
      이번:	49.5425
      사진:	49.5285
     마지막:	47.8300
      운동:	47.2546
      있었:	46.5251
      저녁:	46.3845
      주말:	45.2956
      아직:	45.0918
      그런:	45.0457
      벌써:	44.9831
     오랜만:	44.9244
      커피:	44.8958
      같은:	44.8476
      사실:	44.6148
     갑자기:	44.5535
      마음:	43.4440
      한번:	43.4385
     하지만:	43.2811
      시작:	43.2676
      뭔가:	42.6081
      먹어:	42.1775
     ㅎㅎㅎ:	41.2229
      아니:	41.1481
      보니:	40.3524
      많은:	40.2946
      지금:

##### 명사 키워드 분석
- Konlpy 사용

In [10]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.3 MB/s 
     |████████████████████████████████| 465 kB 51.9 MB/s 


In [11]:
from konlpy.tag import Okt, Kkma

okt = Okt()
kkma = Kkma()

In [9]:
clean_t[345]

'벌써 월이댜닛 년도 얼마 안남았다 미쳐버려 진쨔 오늘도 빠르게 애증의 ㅋㅋㅋㅋㅋㅋ 월 일 부터 월 일까지의 기록 월 일 월요일 필라테스 오전에 갔다오고 저녁에는 밖에서 걷고 배드민턴치기 요즘에 배드민턴에 푹 빠져있어융ㅎㅎ 헬테기와서 이번달에 헬스 번 간건 안비밀 월 일 화요일 화요일도 필라테스 다녀오기 연속은 힘들엉 점심으로 에그드랍 저녁은 반올림피자샾 후다닥 동네 안경점가서 안경맞추기 원래 눈 짱좋았는데 넘 나빠져서 결국 안경 월 일 수요일 목요일은 뇽또블 또 블랙모밀 갔다왔어유 주간일기에 맨날 나오는것같은 기분ㅋㅋㅋ 이번엔 언니랑 다녀왔는데 언니가 짱 좋아함 양산동 헤이에스프레소바 짱 존맛 전대후문 마로와플 쨩 또 저녁에는 배드민턴 월 일 목요일 목요일 저녁에는 남친 만나서 해안식당가서 아구찜이랑 돌솥볶음밥 뿌시기 다 첨단카페 어반피크닉가기 월 일 금요일 이번엔 주 회 필라테스갔댜ㅋㅋㅋ 필라테스 참 힘든데 나름 꾸준히 하는둥 공복으로 갔더니 당떨어져서 이디야 최애메뉴 흑당콜드브루 큰걸로 키키 월 일 토요일 토요일은 외가식구들 다 모여서 곡성에 펜션 잡고 박 일 여행 또 먹고왔댱 빠져있는 별빛청하 사랑훼 월 일 일요일 아침에 일어나서 조카가 라면 끓여줬는데 넘모 넘모 맛있었땽 라면은 김통깨라면쓰 곡성 출렁다리 갔다가 참게수제비맛집 청솔가든갔다가 근처 카페 소나무 다녀왔땨 다들 빠빠이 하구 집으로 왔는데 뭔가 아쉬워서 담양 신상카페 갔다가 저녁은 짜장면으로 하루 마무리 마지막 사진은 귀여운 감나무사진 월 첫째주 주간일기도 끄으으읕 블로그챌린지 월첫째주 월주간일기 일상 일기 뇽뇽일기 일기쓰기 공감 소통'

In [17]:
okt.pos(clean_t[345], join=True)

['벌써/Noun',
 '월이댜닛/Noun',
 '년도/Noun',
 '얼마/Noun',
 '안남았다/Adjective',
 '미쳐/Adjective',
 '버려/Verb',
 '진쨔/Noun',
 '오늘/Noun',
 '도/Josa',
 '빠르게/Adjective',
 '애증/Noun',
 '의/Josa',
 'ㅋㅋㅋㅋㅋㅋ/KoreanParticle',
 '월/Noun',
 '일/Noun',
 '부터/Noun',
 '월/Noun',
 '일/Noun',
 '까지의/Josa',
 '기록/Noun',
 '월/Noun',
 '일/Noun',
 '월요일/Noun',
 '필라테스/Noun',
 '오전/Noun',
 '에/Josa',
 '갔다오고/Verb',
 '저녁/Noun',
 '에는/Josa',
 '밖/Noun',
 '에서/Josa',
 '걷고/Verb',
 '배드민턴/Noun',
 '치기/Noun',
 '요즘/Noun',
 '에/Josa',
 '배드민턴/Noun',
 '에/Josa',
 '푹/Noun',
 '빠져있어/Verb',
 '융/Noun',
 'ㅎㅎ/KoreanParticle',
 '헬테기와/Noun',
 '서/Josa',
 '이번/Noun',
 '달/Noun',
 '에/Josa',
 '헬스/Noun',
 '번/Noun',
 '간건/Verb',
 '안/Noun',
 '비밀/Noun',
 '월/Noun',
 '일/Noun',
 '화요일/Noun',
 '화요일/Noun',
 '도/Josa',
 '필라테스/Noun',
 '다녀오기/Verb',
 '연속/Noun',
 '은/Josa',
 '힘들/Adjective',
 '엉/Exclamation',
 '점심/Noun',
 '으로/Josa',
 '에그드랍/Noun',
 '저녁/Noun',
 '은/Josa',
 '반올림/Noun',
 '피자/Noun',
 '샾/Noun',
 '후다닥/Noun',
 '동네/Noun',
 '안경점/Noun',
 '가서/Verb',
 '안경/Noun',
 '맞추기/Verb',
 '원래/Nou

In [21]:
clean_t_nouns_okt = []
for tt in tqdm(clean_t):
    clean_t_nouns_okt.append(list(set(okt.nouns(tt))))

100%|██████████| 8540/8540 [09:59<00:00, 14.25it/s]


In [22]:
clean_t_nouns_okt_ = []
for tt in clean_t_nouns_okt:
    clean_t_nouns_okt_.append(' '.join(tt))

In [23]:
wordrank_extractor = KRWordRank(min_count = 5, max_length = 10, verbose = True)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(clean_t_nouns_okt_, beta, max_iter)

scan vocabs ... 
num vocabs = 29246
done


In [26]:
for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:1000]:
    print('%8s:\t%.4f' % (word, r))

      일기:	31.0293
      생각:	30.0189
      시간:	25.9023
      사람:	24.8379
      다음:	23.8675
      사진:	22.4255
      이번:	22.1131
      시작:	21.1399
      아이:	20.8116
      하루:	19.4324
      저녁:	19.3611
      마음:	18.2735
      주말:	17.7995
      오늘:	17.6826
      아침:	17.0584
      보고:	16.7897
      여기:	16.2711
      하나:	16.1385
      주간:	16.1017
      린지:	16.0531
      블로:	15.9129
      카페:	15.5692
      지난:	15.3865
      친구:	15.0546
      느낌:	14.5864
      점심:	14.2105
      엄마:	13.9733
      처음:	13.9581
      주차:	13.9559
      마지:	13.9247
      일상:	13.7133
      우리:	13.6109
      가을:	13.5551
      커피:	13.0462
      날씨:	12.7718
      월요:	12.7660
      기분:	12.7316
     금요일:	12.4944
      계속:	12.4461
      운동:	12.3887
      정도:	12.2848
      여행:	12.1682
      포스:	12.1127
     토요일:	11.9867
      출근:	11.6605
      때문:	11.6055
      퇴근:	11.5001
      벌써:	11.3994
      마무:	11.2839
      일요:	11.2095
      준비:	11.2000
      사실:	11.1492
      조금:	11.1139
      지금:	11.0996
      혼자:	10.7494
      맛집:	

- kkma 를 사용하여 보다 정확한 형태소 분석을 진행하려고 하였으나, gpu 문제로 포기

In [13]:
clean_t_nouns_kk = []
for tt in tqdm(clean_t):
    clean_t_nouns_kk.append(list(set(kkma.nouns(tt))))

clean_t_nouns_kk_ = []
for tt in clean_t_nouns_kk:
    clean_t_nouns_kk_.append(' '.join(tt))

wordrank_extractor = KRWordRank(min_count = 5, max_length = 10, verbose = True)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

keywords, rank, graph = wordrank_extractor.extract(clean_t_nouns_kk_, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:1000]:
    print('%8s:\t%.4f' % (word, r))

  6%|▋         | 547/8540 [20:47<5:03:53,  2.28s/it]


java.lang.OutOfMemoryError: ignored

##### 동사 분석
- 마찬가지로 konlpy의 okt 사용

In [36]:
verbs = []
for tt in tqdm(clean_t):
    pos = okt.pos(tt)
    for p in pos:
        if p[1] == 'Verb':
            verbs.append(p[0])
print(verbs)

100%|██████████| 8540/8540 [15:54<00:00,  8.95it/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




In [37]:
len(verbs)

577492

In [38]:
set_verbs = list(set(verbs))
len(set_verbs)

60543

In [39]:
verbs_count = {}
for v in verbs:
    try:
        verbs_count[v] += 1
    except:
        verbs_count[v] = 0
sorted(verbs_count.items(), key=lambda x: x[1], reverse=True)

[('잘', 8449),
 ('한', 7717),
 ('할', 6916),
 ('하는', 6130),
 ('했는데', 5280),
 ('해서', 5093),
 ('해', 4120),
 ('챌', 3976),
 ('했다', 3853),
 ('넘', 3481),
 ('보니', 3435),
 ('가서', 3263),
 ('한다', 2886),
 ('된', 2446),
 ('참', 2370),
 ('하면', 2346),
 ('하기', 2261),
 ('가는', 2191),
 ('들', 2154),
 ('하면서', 2125),
 ('합니다', 2110),
 ('먹어', 2073),
 ('주는', 1991),
 ('먹는', 1935),
 ('먹었다', 1911),
 ('했던', 1849),
 ('하는데', 1834),
 ('하게', 1830),
 ('되는', 1796),
 ('와서', 1769),
 ('먹고', 1752),
 ('와', 1726),
 ('왔다', 1680),
 ('하다', 1656),
 ('나도', 1630),
 ('하지', 1609),
 ('하다가', 1605),
 ('큰', 1597),
 ('해야', 1566),
 ('본', 1490),
 ('보면', 1488),
 ('먹은', 1474),
 ('먹을', 1437),
 ('가', 1430),
 ('따', 1427),
 ('갈', 1401),
 ('갔는데', 1375),
 ('되어', 1367),
 ('했지만', 1367),
 ('하러', 1344),
 ('않고', 1330),
 ('했어요', 1304),
 ('찍어', 1300),
 ('이었다', 1287),
 ('보는', 1274),
 ('하며', 1245),
 ('먹으러', 1237),
 ('만들어', 1201),
 ('갔다가', 1191),
 ('먹었는데', 1151),
 ('가고', 1065),
 ('봤는데', 1058),
 ('써', 1039),
 ('쓰고', 1037),
 ('된다', 1032),
 ('싶다', 1026),
 ('될', 101

##### 시기별 분석

In [10]:
date_t = []
d = 0
for i in range(len(clean_t)//140):
    date_t.append(clean_t[i:i+140])

In [11]:
len(date_t)

61

In [21]:
len(date_t[0])

140

In [22]:
week_tok = []
for i in range(9)
ok = word_tokenize(' '.join(date_t[0]))
ok

['수학',
 '책도',
 '쪽부터',
 '읽는',
 '타입이라',
 '처음에',
 '일기',
 '챌린지를',
 '시작하는',
 '타이밍을',
 '몰라서',
 '그',
 '뒤로',
 '아예',
 '시작을',
 '안',
 '했었다',
 '그러다',
 '결국',
 '시작함',
 'ㅋㅋㅋㅋㅋ',
 '어차피',
 '쓰는',
 '블챌도',
 '써보자',
 '목요일',
 '다음날이',
 '대망의',
 '망고',
 '견학',
 '일이라서',
 '도시락을',
 '싸야',
 '두둥',
 '망고',
 '입학하고',
 '나서는',
 '년간',
 '코로나라서',
 '단',
 '한',
 '번도',
 '소풍',
 '같은',
 '것을',
 '가지',
 '않았기에',
 '도시락',
 '쌀',
 '일이',
 '없었었다',
 '물론',
 '어린이집과',
 '유치원',
 '때',
 '갔었지만',
 '그때는',
 '망고가',
 '김밥',
 '말고',
 '주먹밥이나',
 '샌드위치를',
 '요청했었다',
 '학년이라',
 '김밥을',
 '싸야겠지',
 '사실',
 '김밥을',
 '준비해야',
 '한다는',
 '걸',
 '알고',
 '나서',
 '가장',
 '먼저',
 '한',
 '일은',
 '일단',
 '마켓',
 '컬리에',
 '가서',
 '김밥이',
 '있는지',
 '알아봤고',
 '하지만',
 '전날',
 '말아놓은',
 '것이라서',
 '굉장히',
 '고민이',
 '되었다',
 '집',
 '주변에',
 '김밥',
 '집이',
 '있는지',
 '서칭을',
 '해본',
 '것이었다',
 '없었다',
 '가장',
 '가까운',
 '곳이',
 '동탄에',
 '있었다',
 '결정적으로',
 '망고',
 '선생님께서',
 '산',
 '김밥은',
 '식중독',
 '위험이',
 '있으니',
 '집에서',
 '만들어',
 '오도록',
 '하세요',
 '라고',
 '하셨다고',
 '집에서',
 '만들기로',
 '마음을',
 '굳혔다',
 '목요일인',
 '전날',
 '재료를',
 '잔뜩',
 '구입했다',


In [12]:
date_keyword = []
for i in range(len(date_t)):
    wordrank_extractor = KRWordRank(min_count = 3, max_length = 10, verbose = True)

    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 10

    keywords, rank, graph = wordrank_extractor.extract(date_t[i], beta, max_iter)
    date_keyword.append(keywords)

scan vocabs ... 
num vocabs = 8302
done
scan vocabs ... 
num vocabs = 8277
done
scan vocabs ... 
num vocabs = 8342
done
scan vocabs ... 
num vocabs = 8323
done
scan vocabs ... 
num vocabs = 8240
done
scan vocabs ... 
num vocabs = 8268
done
scan vocabs ... 
num vocabs = 8283
done
scan vocabs ... 
num vocabs = 8282
done
scan vocabs ... 
num vocabs = 8320
done
scan vocabs ... 
num vocabs = 8394
done
scan vocabs ... 
num vocabs = 8365
done
scan vocabs ... 
num vocabs = 8356
done
scan vocabs ... 
num vocabs = 8325
done
scan vocabs ... 
num vocabs = 8338
done
scan vocabs ... 
num vocabs = 8302
done
scan vocabs ... 
num vocabs = 8285
done
scan vocabs ... 
num vocabs = 8241
done
scan vocabs ... 
num vocabs = 8171
done
scan vocabs ... 
num vocabs = 8204
done
scan vocabs ... 
num vocabs = 8219
done
scan vocabs ... 
num vocabs = 8210
done
scan vocabs ... 
num vocabs = 8249
done
scan vocabs ... 
num vocabs = 8240
done
scan vocabs ... 
num vocabs = 8209
done
scan vocabs ... 
num vocabs = 8256
done


In [23]:
len(date_keyword)

61

In [31]:
date_keyword[49:56]

[{'맛있': 10.57924278151331,
  '오늘은': 7.528058064098249,
  '생각': 6.689289843481774,
  '다음': 6.444495813567877,
  '시작': 6.110568745700069,
  '주차': 6.014491983570554,
  '이번': 5.929050755401259,
  '엄마': 5.872855807340479,
  '운동': 5.7905568475104765,
  '좋아': 5.645634370219108,
  '하루': 5.592310775959094,
  '벌써': 5.490358967132956,
  '안녕하세요': 5.388248359122574,
  '먹었': 5.2301220586172965,
  '동영상': 5.1315821204345005,
  '다른': 4.8536632644891675,
  '한다': 4.776335607353594,
  '처음': 4.7493215397406825,
  'ㅋㅋㅋ': 4.745726229578474,
  '시간': 4.527758168287924,
  '들어': 4.5242871325080385,
  '여기': 4.496002000020692,
  '먹어': 4.495112798313655,
  '있었': 4.465148074118138,
  '다녀': 4.443224452335758,
  '아직': 4.3704637909055855,
  '저녁': 4.3428778076147445,
  '친구': 4.33468714835987,
  '월요일': 4.270378881983083,
  '했는데': 4.240787151358487,
  '아침에': 4.165839486854579,
  '먹는': 4.162038603501506,
  '사람': 4.04306809752629,
  '일기': 4.026245745763884,
  '조금': 4.025820393526939,
  '드디어': 3.956240873406166,
  '보고': 3.93

##### 연어 분석

In [18]:
from nltk import ngrams
from nltk import collocations
from konlpy.utils import pprint

In [19]:
len(clean_t)

8540

In [20]:
total_clean_t = ' '.join(clean_t)

In [21]:
bigram_measures = collocations.BigramAssocMeasures() 
text_pos = okt.pos(total_clean_t)
bigrams = collocations.BigramCollocationFinder.from_words(text_pos)
bigrams.apply_freq_filter(5) # 최소 5번 등장하는 n-gram 필터링

In [27]:
pprint(bigrams.nbest(bigram_measures.pmi, 500))

[(('고르자', 'Verb'), ('브종', 'Noun')),
 (('몽마', 'Noun'), ('르뜨', 'Noun')),
 (('종파', 'Noun'), ('르코', 'Noun')),
 (('프루토', 'Noun'), ('프루타', 'Noun')),
 (('맑스', 'Noun'), ('엥겔스', 'Noun')),
 (('오니온', 'Noun'), ('앤온', 'Noun')),
 (('고군', 'Adjective'), ('분투', 'Noun')),
 (('문경시', 'Noun'), ('문경읍', 'Noun')),
 (('통영시', 'Noun'), ('광도면', 'Noun')),
 (('가내', 'Noun'), ('수공업', 'Noun')),
 (('산타마리아', 'Noun'), ('노벨라', 'Noun')),
 (('안노', 'Verb'), ('히데아키', 'Noun')),
 (('오르락', 'Noun'), ('내리락', 'Noun')),
 (('피톤', 'Noun'), ('치드', 'Noun')),
 (('골격', 'Noun'), ('근량', 'Noun')),
 (('붙박', 'Verb'), ('이장', 'Noun')),
 (('듀퐁', 'Noun'), ('서클', 'Noun')),
 (('쓸줄', 'Verb'), ('어드벤쳐', 'Noun')),
 (('드트', 'Noun'), ('래커', 'Noun')),
 (('부', 'Adjective'), ('탁해', 'Adjective')),
 (('양산시', 'Noun'), ('물금읍', 'Noun')),
 (('졌', 'Verb'), ('잘싸', 'Verb')),
 (('바이킹스', 'Noun'), ('워프', 'Noun')),
 (('센과', 'Verb'), ('치히로', 'Noun')),
 (('주암면', 'Noun'), ('행정리', 'Noun')),
 (('엠버', 'Noun'), ('서더', 'Verb')),
 (('구례군', 'Noun'), ('구례읍', 'Noun')),
 (('스올', 'Nou

In [40]:
col = bigrams.nbest(bigram_measures.pmi, 500)
col[0]

(('고르자', 'Verb'), ('브종', 'Noun'))

In [41]:
col = [' '.join([a[0], b[0]]) for a, b in col]
col[0]

'고르자 브종'

In [43]:
col_dic = {}
for tt in tqdm(clean_t):
    for c in col:
        try:
            col_dic[c] += tt.count(c)
        except:
            col_dic[c] = tt.count(c)

100%|██████████| 8540/8540 [00:07<00:00, 1148.25it/s]


- 연어를 바탕으로 많이 나타난 단어를 찾고자 하였음. 연어의 지수가 높아도 현재의 유행을 담고있다고 보기는 어려워 보고서에 넣지는 않았음.

In [45]:
sorted(col_dic.items(), key=lambda x: x[1], reverse=True)

[('성남시 분당구', 32),
 ('캐모마일 릴렉', 27),
 ('발랄 티아', 26),
 ('경주시 강동면', 25),
 ('캐모마일 릴렉서', 25),
 ('수원시 영통구', 25),
 ('손발 닦이고', 25),
 ('아니나 다를까', 23),
 ('본의 아니게', 21),
 ('용인시 기흥구', 20),
 ('천왕성 엄폐', 19),
 ('짧고 굵게', 19),
 ('고픈 낙서장', 18),
 ('수원시 팔달구', 16),
 ('쿠키런 킹덤', 15),
 ('전라북도 전주시', 15),
 ('부산진구 서전로', 15),
 ('충청남도 천안시', 14),
 ('대전광역시 유성구', 14),
 ('전라북도 완주군', 13),
 ('해운대구 해운대로', 13),
 ('전주시 완산구', 12),
 ('용인시 처인구', 12),
 ('용산구 이태원로', 12),
 ('강동면 강동로', 11),
 ('춥지도 덥지도', 11),
 ('분당구 판교역', 11),
 ('프랑코 폰타나', 10),
 ('미산 호랭이', 10),
 ('정약용 유적지', 10),
 ('듀오 링고', 10),
 ('척추 세우기', 10),
 ('다산 정약용', 10),
 ('커닝 방지', 10),
 ('알라딘 중고서점', 10),
 ('영등포구 여의대로', 10),
 ('충청북도 청주시', 10),
 ('천안시 서북구', 9),
 ('살려면 돌아다녀야', 9),
 ('스칼렛 바이올렛', 9),
 ('만들어졌으니 무단', 9),
 ('강서구 화곡로', 9),
 ('미드나잇 액터', 8),
 ('네트워킹 밀레니엄', 8),
 ('레오 버넷', 8),
 ('역마살 살려면', 8),
 ('밀레니엄 선글라스', 8),
 ('부탁드립니다 작성자', 8),
 ('틀어진 척추', 8),
 ('무단 복제', 8),
 ('강서구 공항대로', 8),
 ('덧글 부탁드립니다', 8),
 ('광진구 능동', 8),
 ('일산동구 정발', 8),
 ('충청북도 제천시', 8),
 ('유튜버 알리페', 8),
 ('

In [26]:
bigram_measures = collocations.BigramAssocMeasures() 
# trigram_measures = collocations.TrigramAssocMeasures()
# fourgram_measures = collocations.QuadgramAssocMeasures()

text_pos = okt.pos(total_clean_t)

bigrams = collocations.BigramCollocationFinder.from_words(text_pos)
bigrams.apply_freq_filter(5) # 최소 5번 등장하는 n-gram 필터링
pprint(bigrams.nbest(bigram_measures.pmi, 20)) # PMI 가 높은 상위 N개의 n-gram을 collocation 으로 추출

[(('고르자', 'Verb'), ('브종', 'Noun')),
 (('몽마', 'Noun'), ('르뜨', 'Noun')),
 (('종파', 'Noun'), ('르코', 'Noun')),
 (('프루토', 'Noun'), ('프루타', 'Noun')),
 (('맑스', 'Noun'), ('엥겔스', 'Noun')),
 (('오니온', 'Noun'), ('앤온', 'Noun')),
 (('고군', 'Adjective'), ('분투', 'Noun')),
 (('문경시', 'Noun'), ('문경읍', 'Noun')),
 (('통영시', 'Noun'), ('광도면', 'Noun')),
 (('가내', 'Noun'), ('수공업', 'Noun')),
 (('산타마리아', 'Noun'), ('노벨라', 'Noun')),
 (('안노', 'Verb'), ('히데아키', 'Noun')),
 (('오르락', 'Noun'), ('내리락', 'Noun')),
 (('피톤', 'Noun'), ('치드', 'Noun')),
 (('골격', 'Noun'), ('근량', 'Noun')),
 (('붙박', 'Verb'), ('이장', 'Noun')),
 (('듀퐁', 'Noun'), ('서클', 'Noun')),
 (('쓸줄', 'Verb'), ('어드벤쳐', 'Noun')),
 (('드트', 'Noun'), ('래커', 'Noun')),
 (('부', 'Adjective'), ('탁해', 'Adjective'))]


In [46]:
trigram_measures = collocations.TrigramAssocMeasures()
# fourgram_measures = collocations.QuadgramAssocMeasures()

trigrams = collocations.TrigramCollocationFinder.from_words(text_pos)
trigrams.apply_freq_filter(5) # 최소 5번 등장하는 n-gram 필터링
col_tri = trigrams.nbest(trigram_measures.pmi, 500) # PMI 가 높은 상위 N개의 n-gram을 collocation 으로 추출

In [49]:
col_tri = [' '.join([a[0], b[0], c[0]]) for a, b, c in col_tri]
col_tri_dic = {}
for tt in tqdm(clean_t):
    for c in col_tri:
        try:
            col_tri_dic[c] += tt.count(c)
        except:
            col_tri_dic[c] = tt.count(c)
sorted(col_tri_dic.items(), key=lambda x: x[1], reverse=True)

100%|██████████| 8540/8540 [00:06<00:00, 1381.22it/s]


[('걸은 걸이 걸음', 35),
 ('경기도 성남시 분당구', 31),
 ('경기도 고양시 일산동구', 27),
 ('경상북도 경주시 강동면', 25),
 ('경기도 수원시 영통구', 25),
 ('서울특별시 송파구 올림픽로', 23),
 ('경기도 용인시 기흥구', 20),
 ('경기도 수원시 팔달구', 16),
 ('엑스포 진주 유등', 16),
 ('부산광역시 부산진구 서전로', 15),
 ('송파구 올림픽로 롯데', 15),
 ('부산진구 서전로 번길', 14),
 ('부산광역시 해운대구 해운대로', 13),
 ('해운대구 해운대로 번길', 13),
 ('오로라 캐모마일 릴렉', 12),
 ('고양시 덕양구 고양', 12),
 ('서울특별시 용산구 이태원로', 12),
 ('서울특별시 용산구 한강대로', 12),
 ('전라북도 전주시 완산구', 11),
 ('경주시 강동면 강동로', 11),
 ('성남시 분당구 판교역', 11),
 ('믹 베티 리카', 11),
 ('특별자치도 제주시 애월읍', 11),
 ('경기도 용인시 처인구', 11),
 ('다산 정약용 유적지', 10),
 ('궁금할 땐 톡톡하세요', 10),
 ('커닝 방지 모자', 10),
 ('서울특별시 영등포구 여의대로', 10),
 ('서울특별시 마포구 와우산', 10),
 ('대학 커닝 방지', 9),
 ('광양 백운산 둘레길', 9),
 ('수원 화성 행궁', 9),
 ('부산광역시 해운대구 달맞이', 9),
 ('필리핀 대학 커닝', 9),
 ('살려면 돌아다녀야 한다', 9),
 ('역마살 살려면 돌아다녀야', 8),
 ('덧글 부탁드립니다 작성자', 8),
 ('충청남도 천안시 서북구', 8),
 ('만들어라 레오 버넷', 8),
 ('아침밥 네트워킹 밀레니엄', 8),
 ('강동면 강동로 강동', 8),
 ('고양시 일산동구 정발', 8),
 ('무단 복제 금지', 8),
 ('달려오시느라 정말로 정말로', 8),
 ('만들어라 재미있게 만들어라', 8),
 ('아닙니다 부

In [28]:
trigram_measures = collocations.TrigramAssocMeasures()
# fourgram_measures = collocations.QuadgramAssocMeasures()

text_pos = okt.pos(total_clean_t)

trigrams = collocations.TrigramCollocationFinder.from_words(text_pos)
trigrams.apply_freq_filter(5) # 최소 5번 등장하는 n-gram 필터링
pprint(trigrams.nbest(trigram_measures.pmi, 20)) # PMI 가 높은 상위 N개의 n-gram을 collocation 으로 추출

[(('아림', 'Adjective'), ('클로', 'Verb'), ('젯', 'Noun')),
 (('미드나잇', 'Noun'), ('액터', 'Noun'), ('뮤지션', 'Noun')),
 (('경성', 'Noun'), ('연쇄', 'Noun'), ('실종', 'Noun')),
 (('츠루', 'Noun'), ('마키', 'Noun'), ('카즈', 'Noun')),
 (('가타', 'Noun'), ('메구미', 'Noun'), ('하야시', 'Noun')),
 (('주암면', 'Noun'), ('행정리', 'Noun'), ('번지', 'Noun')),
 (('맑스', 'Noun'), ('엥겔스', 'Noun'), ('연구소', 'Noun')),
 (('더티', 'Noun'), ('노프', 'Noun'), ('헬로', 'Noun')),
 (('노보텔', 'Noun'), ('앰', 'Noun'), ('배서', 'Noun')),
 (('내셔널', 'Noun'), ('지오', 'Noun'), ('그래픽', 'Noun')),
 (('라네즈', 'Noun'), ('메종', 'Noun'), ('키츠네', 'Noun')),
 (('역마살', 'Noun'), ('살려면', 'Verb'), ('돌아다녀야', 'Verb')),
 (('네트워킹', 'Noun'), ('밀레니엄', 'Noun'), ('선글라스', 'Noun')),
 (('에로', 'Josa'), ('더꽃', 'Noun'), ('작업실', 'Noun')),
 (('경상북도', 'Noun'), ('문경시', 'Noun'), ('문경읍', 'Noun')),
 (('어드', 'Noun'), ('벤트', 'Noun'), ('캘린더', 'Noun')),
 (('덧글', 'Noun'), ('부탁드립니다', 'Adjective'), ('작성자', 'Noun')),
 (('국립공원', 'Noun'), ('스탬', 'Noun'), ('프투', 'Noun')),
 (('덕산', 'Noun'), ('스플', 'Noun'), ('

In [50]:
fourgram_measures = collocations.QuadgramAssocMeasures()

fourgrams = collocations.QuadgramCollocationFinder.from_words(text_pos)
fourgrams.apply_freq_filter(5) # 최소 5번 등장하는 n-gram 필터링
col_four = fourgrams.nbest(fourgram_measures.pmi, 500) # PMI 가 높은 상위 N개의 n-gram을 collocation 으로 추출

In [51]:
col_four = [' '.join([a[0], b[0], c[0], d[0]]) for a, b, c, d in col_four]
col_four_dic = {}
for tt in tqdm(clean_t):
    for c in col_four:
        try:
            col_four_dic[c] += tt.count(c)
        except:
            col_four_dic[c] = tt.count(c)
sorted(col_four_dic.items(), key=lambda x: x[1], reverse=True)

100%|██████████| 8540/8540 [00:05<00:00, 1545.59it/s]


[('성공 걸은 걸이 걸음', 21),
 ('수면시간 분 수면 스코어', 21),
 ('걷기 성공 걸은 걸이', 21),
 ('레깅스 사이즈 삭스 나이키', 17),
 ('부산광역시 부산진구 서전로 번길', 14),
 ('장 경상북도 경주시 강동면', 14),
 ('실패 걸은 걸이 걸음', 14),
 ('걷기 실패 걸은 걸이', 14),
 ('꿀뷔 시 쓰고 읽어주는', 14),
 ('점 경기도 성남시 분당구', 14),
 ('부산광역시 해운대구 해운대로 번길', 13),
 ('서울특별시 송파구 올림픽로 롯데', 13),
 ('경상북도 경주시 강동면 강동로', 11),
 ('경기도 성남시 분당구 판교역', 11),
 ('경기도 고양시 덕양구 고양', 10),
 ('부산진구 서전로 번길 층', 10),
 ('할로윈 초코 헤이즐넛 프라푸치노', 10),
 ('안타깝다 필리핀 대학 커닝', 9),
 ('필리핀 대학 커닝 방지', 9),
 ('살려면 돌아다녀야 한다 돌아다니는', 9),
 ('대학 커닝 방지 모자', 9),
 ('수원 화성 행궁 여행', 9),
 ('주차 수원 화성 행궁', 9),
 ('돌이 굿 아이디어 이렇게', 9),
 ('경주시 강동면 강동로 강동', 8),
 ('서 역마살 살려면 돌아다녀야', 8),
 ('도용 리뷰 무단 복제', 8),
 ('역마살 살려면 돌아다녀야 한다', 8),
 ('만들어라 재미있게 만들어라 시선', 8),
 ('달려오시느라 정말로 정말로 수고', 8),
 ('경기도 고양시 일산동구 정발', 8),
 ('게 만들어라 레오 버넷', 8),
 ('새별 드림 신간 새별', 8),
 ('리뷰 무단 복제 금지', 8),
 ('되셨다면 공감 꼭 부탁드리구요', 8),
 ('청담 헤어 아티스트 지은', 8),
 ('무단 복제 금지 내', 8),
 ('는 덧글 부탁드립니다 작성자', 8),
 ('연락처 경기 고양시 덕양구', 8),
 ('흐름 및 대출 청약', 8),
 ('즐겁게 보셨다면 좋겠습니다 감사합니다', 8),
 ('스러운 새별

In [13]:
n = 2
sent = '맛은 훌륭했지만 대신 서비스가 최악이었다.'
morph = okt.morphs(sent)

ngram = ngrams(morph, n)
for gram in ngram:
  print(gram)

('맛', '은')
('은', '훌륭했지만')
('훌륭했지만', '대신')
('대신', '서비스')
('서비스', '가')
('가', '최악')
('최악', '이었다')
('이었다', '.')
